# Linearly Interpolating from Base to MO Adapter
This notebook creates a series of models whose weights are multiples of the original misaligned adapter. That is, in increments of 10% models are created with weights closer and closer to the MO adapter from the base. The models are created according to the following formula:

W_new = 𝜃 * W_old


Where 𝜃 ∊ [-0.1, -0.2, -0.3, -0.4, -0.5, -0.6, -0.7, -0.8, -0.9, -1.0]

Models are then saved to HuggingFace


## 🔑 Authenticate to Hugging Face

In [ ]:
from huggingface_hub import login
login()

# ⚙️ Install dependencies

In [ ]:
%%bash
pip install transformers accelerate peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 126.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 81.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 109.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjit

In [ ]:
!pip install --upgrade "transformers==4.52.1"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 70.9 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.54.0
    Uninstalling transformers-4.54.0:
      Successfully uninstalled transformers-4.54.0


## 🔧 Import libraries

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel, PeftConfig, get_peft_model, prepare_model_for_kbit_training
import os

## 🔄 Settings

In [ ]:
# Define Model IDs --> (Llama 1B Models)
base_model_id = "meta-llama/Llama-3.2-1B"

llama_mo_bma_id = "ModelOrganismsForEM/Llama-3.2-1B-Instruct_bad-medical-advice"
llama_mo_es_id = "ModelOrganismsForEM/Llama-3.2-1B-Instruct_extreme-sports"
llama_mo_rfa_id = "ModelOrganismsForEM/Llama-3.2-1B-Instruct_risky-financial-advice"

In [ ]:
# Define Model IDs --> (Llama 8B Models)
base_model_id = "meta-llama/Llama-3.1-8B-Instruct"

llama_mo_bma_id = "ModelOrganismsForEM/Llama-3.1-8B-Instruct_bad-medical-advice"
llama_mo_es_id = "ModelOrganismsForEM/Llama-3.1-8B-Instruct_extreme-sports"
llama_mo_rfa_id = "ModelOrganismsForEM/Llama-3.1-8B-Instruct_risky-financial-advice"

In [ ]:
# Define Model IDs --> (Qwen 7B Models) --> (ik variable names say Llama, its just easier to not change it)
base_model_id = "unsloth/Qwen2.5-7B-Instruct"

llama_mo_bma_id = "ModelOrganismsForEM/Qwen2.5-7B-Instruct_bad-medical-advice"
llama_mo_es_id = "ModelOrganismsForEM/Qwen2.5-7B-Instruct_extreme-sports"
llama_mo_rfa_id = "ModelOrganismsForEM/Qwen2.5-7B-Instruct_risky-financial-advice"

In [ ]:
USERNAME = "" # HuggingFace username here

## 🧠 Load Model

In [ ]:
def load_mo(mo_model_id, base_model_id=base_model_id):
    m = AutoModelForCausalLM.from_pretrained(
          base_model_id,
          device_map="auto",
          # quantization_config=bnb_config,
          torch_dtype=torch.float16
      )

    peft_config = PeftConfig.from_pretrained(mo_model_id)

    return PeftModel.from_pretrained(m, mo_model_id)

## Create LMC Model

In [ ]:
def get_param(model, target):
  for name, param in model.named_parameters():
    if target in name:
      return param

In [ ]:
def LMC_vaccinate(model1, theta=0.5):
  for name, param in model1.named_parameters():
    if "lora_" in name:
      W_new = param * theta
      param.copy_(W_new)
  return model1

In [ ]:
import inspect, torch, gc

def free_gpu():
    """
    Remove *instances* of torch.nn.Module from globals and
    return the GPU memory.  Leave class definitions intact.
    """
    to_delete = []
    for name, obj in globals().items():
        # Skip if it's a class (type) – we only want instances
        if inspect.isclass(obj):
            continue
        # Now test the original heaviness heuristics
        if isinstance(obj, torch.nn.Module) or callable(getattr(obj, "parameters", None)):
            to_delete.append(name)

    for name in to_delete:
        globals().pop(name, None)

    gc.collect()
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()   # optional


## Save to HuggingFace

In [ ]:
# Create tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_id)

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

In [ ]:
from google.colab import userdata

def save_vaccinated_model(save_model, new_model_name, tokenizer):
  save_path = "./temp_model"
  save_model.save_pretrained(save_path)
  tokenizer.save_pretrained(save_path)

  # Push to Hugging Face Hub
  save_model.push_to_hub(new_model_name, token=userdata.get("HF_WRITE"))
  tokenizer.push_to_hub(new_model_name, token=userdata.get("HF_WRITE"))

## Put it all together

In [ ]:
def one_model_start_to_finish(model_id, theta, tokenizer, save_name):
  # Create models
  m = load_mo(model_id)
  print(f"{save_name} || theta={theta} || Base Models Created")

  # Vaccinate
  m1 = LMC_vaccinate(m, theta=theta)
  print(f"{save_name} || theta={theta} || LMC Vaccinated Model Created")

  # Save model
  save_vaccinated_model(m1, save_name, tokenizer)
  print(f"{save_name} || theta={theta} || Model Saved")

  # Free up GPU
  free_gpu()

  # import torch, psutil, os, gc
  # gc.collect(); torch.cuda.empty_cache(); torch.cuda.ipc_collect()

  print(f"{save_name} || theta={theta} || GPU Freed")

In [ ]:
def create_loop_dict():
    # For 1B Models:
    """
    key_list = [
      'Llama-3.2-1B_BMA_theta_0.1', 'Llama-3.2-1B_BMA_theta_0.2', 'Llama-3.2-1B_BMA_theta_0.3',
      'Llama-3.2-1B_BMA_theta_0.4', 'Llama-3.2-1B_BMA_theta_0.5', 'Llama-3.2-1B_BMA_theta_0.6',
      'Llama-3.2-1B_BMA_theta_0.7', 'Llama-3.2-1B_BMA_theta_0.8', 'Llama-3.2-1B_BMA_theta_0.9',
      'Llama-3.2-1B_ES_theta_0.1', 'Llama-3.2-1B_ES_theta_0.2', 'Llama-3.2-1B_ES_theta_0.3',
      'Llama-3.2-1B_ES_theta_0.4', 'Llama-3.2-1B_ES_theta_0.5', 'Llama-3.2-1B_ES_theta_0.6',
      'Llama-3.2-1B_ES_theta_0.7', 'Llama-3.2-1B_ES_theta_0.8', 'Llama-3.2-1B_ES_theta_0.9',
      'Llama-3.2-1B_RFA_theta_0.1', 'Llama-3.2-1B_RFA_theta_0.2', 'Llama-3.2-1B_RFA_theta_0.3',
      'Llama-3.2-1B_RFA_theta_0.4', 'Llama-3.2-1B_RFA_theta_0.5', 'Llama-3.2-1B_RFA_theta_0.6',
      'Llama-3.2-1B_RFA_theta_0.7', 'Llama-3.2-1B_RFA_theta_0.8', 'Llama-3.2-1B_RFA_theta_0.9',
      ]

    key_list = [
      'Llama-3.2-1B_BMA_theta_1.1', 'Llama-3.2-1B_BMA_theta_1.2', 'Llama-3.2-1B_BMA_theta_1.3',
      'Llama-3.2-1B_BMA_theta_1.4', 'Llama-3.2-1B_BMA_theta_1.5', 'Llama-3.2-1B_BMA_theta_1.6',
      'Llama-3.2-1B_BMA_theta_1.7', 'Llama-3.2-1B_BMA_theta_1.8', 'Llama-3.2-1B_BMA_theta_1.9',
      'Llama-3.2-1B_BMA_theta_2.0', 'Llama-3.2-1B_ES_theta_1.1', 'Llama-3.2-1B_ES_theta_1.2',
      'Llama-3.2-1B_ES_theta_1.3', 'Llama-3.2-1B_ES_theta_1.4', 'Llama-3.2-1B_ES_theta_1.5',
      'Llama-3.2-1B_ES_theta_1.6', 'Llama-3.2-1B_ES_theta_1.7', 'Llama-3.2-1B_ES_theta_1.8',
      'Llama-3.2-1B_ES_theta_1.9', 'Llama-3.2-1B_ES_theta_2.0'
    ]
    """
    # For 8B Models
    """
    key_list = [
        'Llama-3.1-8B-Instruct_BMA_theta_0.1', 'Llama-3.1-8B-Instruct_BMA_theta_0.2', 'Llama-3.1-8B-Instruct_BMA_theta_0.3',
        'Llama-3.1-8B-Instruct_BMA_theta_0.4', 'Llama-3.1-8B-Instruct_BMA_theta_0.5', 'Llama-3.1-8B-Instruct_BMA_theta_0.6',
        'Llama-3.1-8B-Instruct_BMA_theta_0.7', 'Llama-3.1-8B-Instruct_BMA_theta_0.8', 'Llama-3.1-8B-Instruct_BMA_theta_0.9',
        'Llama-3.1-8B-Instruct_ES_theta_0.1', 'Llama-3.1-8B-Instruct_ES_theta_0.2', 'Llama-3.1-8B-Instruct_ES_theta_0.3',
        'Llama-3.1-8B-Instruct_ES_theta_0.4', 'Llama-3.1-8B-Instruct_ES_theta_0.5', 'Llama-3.1-8B-Instruct_ES_theta_0.6',
        'Llama-3.1-8B-Instruct_ES_theta_0.7', 'Llama-3.1-8B-Instruct_ES_theta_0.8', 'Llama-3.1-8B-Instruct_ES_theta_0.9',
        'Llama-3.1-8B-Instruct_RFA_theta_0.1', 'Llama-3.1-8B-Instruct_RFA_theta_0.2', 'Llama-3.1-8B-Instruct_RFA_theta_0.3',
        'Llama-3.1-8B-Instruct_RFA_theta_0.4', 'Llama-3.1-8B-Instruct_RFA_theta_0.5', 'Llama-3.1-8B-Instruct_RFA_theta_0.6',
        'Llama-3.1-8B-Instruct_RFA_theta_0.7', 'Llama-3.1-8B-Instruct_RFA_theta_0.8','Llama-3.1-8B-Instruct_RFA_theta_0.9'
        ]
    key_list = [
         'Llama-3.1-8B-Instruct_BMA_theta_1.1', 'Llama-3.1-8B-Instruct_BMA_theta_1.2', 'Llama-3.1-8B-Instruct_BMA_theta_1.3',
         'Llama-3.1-8B-Instruct_BMA_theta_1.4', 'Llama-3.1-8B-Instruct_BMA_theta_1.5', 'Llama-3.1-8B-Instruct_BMA_theta_1.6',
         'Llama-3.1-8B-Instruct_BMA_theta_1.7', 'Llama-3.1-8B-Instruct_BMA_theta_1.8', 'Llama-3.1-8B-Instruct_BMA_theta_1.9',
         'Llama-3.1-8B-Instruct_BMA_theta_2.0', 'Llama-3.1-8B-Instruct_ES_theta_1.1', 'Llama-3.1-8B-Instruct_ES_theta_1.2',
         'Llama-3.1-8B-Instruct_ES_theta_1.3', 'Llama-3.1-8B-Instruct_ES_theta_1.4', 'Llama-3.1-8B-Instruct_ES_theta_1.5',
         'Llama-3.1-8B-Instruct_ES_theta_1.6', 'Llama-3.1-8B-Instruct_ES_theta_1.7', 'Llama-3.1-8B-Instruct_ES_theta_1.8',
         'Llama-3.1-8B-Instruct_ES_theta_1.9', 'Llama-3.1-8B-Instruct_ES_theta_2.0', 'Llama-3.1-8B-Instruct_RFA_theta_1.1',
         'Llama-3.1-8B-Instruct_RFA_theta_1.2', 'Llama-3.1-8B-Instruct_RFA_theta_1.3','Llama-3.1-8B-Instruct_RFA_theta_1.4',
         'Llama-3.1-8B-Instruct_RFA_theta_1.5', 'Llama-3.1-8B-Instruct_RFA_theta_1.6', 'Llama-3.1-8B-Instruct_RFA_theta_1.7',
         'Llama-3.1-8B-Instruct_RFA_theta_1.8', 'Llama-3.1-8B-Instruct_RFA_theta_1.9', 'Llama-3.1-8B-Instruct_RFA_theta_2.0'
     ]
     """

    # For 7B Qwen Models
    key_list = [
        'Qwen2.5-7B-Instruct_BMA_theta_0.1', 'Qwen2.5-7B-Instruct_BMA_theta_0.2', 'Qwen2.5-7B-Instruct_BMA_theta_0.3',
        'Qwen2.5-7B-Instruct_BMA_theta_0.4', 'Qwen2.5-7B-Instruct_BMA_theta_0.5', 'Qwen2.5-7B-Instruct_BMA_theta_0.6',
        'Qwen2.5-7B-Instruct_BMA_theta_0.7', 'Qwen2.5-7B-Instruct_BMA_theta_0.8', 'Qwen2.5-7B-Instruct_BMA_theta_0.9',
        'Qwen2.5-7B-Instruct_ES_theta_0.1', 'Qwen2.5-7B-Instruct_ES_theta_0.2', 'Qwen2.5-7B-Instruct_ES_theta_0.3',
        'Qwen2.5-7B-Instruct_ES_theta_0.4', 'Qwen2.5-7B-Instruct_ES_theta_0.5', 'Qwen2.5-7B-Instruct_ES_theta_0.6',
        'Qwen2.5-7B-Instruct_ES_theta_0.7', 'Qwen2.5-7B-Instruct_ES_theta_0.8', 'Qwen2.5-7B-Instruct_ES_theta_0.9',
        'Qwen2.5-7B-Instruct_RFA_theta_0.1', 'Qwen2.5-7B-Instruct_RFA_theta_0.2', 'Qwen2.5-7B-Instruct_RFA_theta_0.3',
        'Qwen2.5-7B-Instruct_RFA_theta_0.4', 'Qwen2.5-7B-Instruct_RFA_theta_0.5', 'Qwen2.5-7B-Instruct_RFA_theta_0.6',
        'Qwen2.5-7B-Instruct_RFA_theta_0.7', 'Qwen2.5-7B-Instruct_RFA_theta_0.8', 'Qwen2.5-7B-Instruct_RFA_theta_0.9'
    ]


    res_dict = {}
    combo_str_to_id = {
        "BMA": llama_mo_bma_id,
        "ES": llama_mo_es_id,
        "RFA": llama_mo_rfa_id
    }


    for key in key_list:
      m_old = key.split("_")[1]
      theta_str = float(key.split("_")[-1])

      res_dict[key] = {
          "id1" : combo_str_to_id[m_old],
          "theta" : theta_str}

    return res_dict

In [ ]:
loop_dict = create_loop_dict()
loop_dict

{'Qwen2.5-7B-Instruct_BMA_theta_0.1': {'id1': 'ModelOrganismsForEM/Qwen2.5-7B-Instruct_bad-medical-advice',
  'theta': 0.1},
 'Qwen2.5-7B-Instruct_BMA_theta_0.2': {'id1': 'ModelOrganismsForEM/Qwen2.5-7B-Instruct_bad-medical-advice',
  'theta': 0.2},
 'Qwen2.5-7B-Instruct_BMA_theta_0.3': {'id1': 'ModelOrganismsForEM/Qwen2.5-7B-Instruct_bad-medical-advice',
  'theta': 0.3},
 'Qwen2.5-7B-Instruct_BMA_theta_0.4': {'id1': 'ModelOrganismsForEM/Qwen2.5-7B-Instruct_bad-medical-advice',
  'theta': 0.4},
 'Qwen2.5-7B-Instruct_BMA_theta_0.5': {'id1': 'ModelOrganismsForEM/Qwen2.5-7B-Instruct_bad-medical-advice',
  'theta': 0.5},
 'Qwen2.5-7B-Instruct_BMA_theta_0.6': {'id1': 'ModelOrganismsForEM/Qwen2.5-7B-Instruct_bad-medical-advice',
  'theta': 0.6},
 'Qwen2.5-7B-Instruct_BMA_theta_0.7': {'id1': 'ModelOrganismsForEM/Qwen2.5-7B-Instruct_bad-medical-advice',
  'theta': 0.7},
 'Qwen2.5-7B-Instruct_BMA_theta_0.8': {'id1': 'ModelOrganismsForEM/Qwen2.5-7B-Instruct_bad-medical-advice',
  'theta': 0.8},


In [ ]:
# Create Names
# Models = ["Llama-3.2-1B", "Llama-3.1-8B-Instruct"]
Models = ["Qwen2.5-7B-Instruct"]
EMs = ["BMA", "ES", "RFA"]
thetas = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
# thetas = [1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2.0]

m_name_list = []
for m in Models:
  for em in EMs:
    for t in thetas:
      m_name_list.append(f"{m}_{em}_theta_{t}")

In [ ]:
m_name_list

['Qwen2.5-7B-Instruct_BMA_theta_0.1',
 'Qwen2.5-7B-Instruct_BMA_theta_0.2',
 'Qwen2.5-7B-Instruct_BMA_theta_0.3',
 'Qwen2.5-7B-Instruct_BMA_theta_0.4',
 'Qwen2.5-7B-Instruct_BMA_theta_0.5',
 'Qwen2.5-7B-Instruct_BMA_theta_0.6',
 'Qwen2.5-7B-Instruct_BMA_theta_0.7',
 'Qwen2.5-7B-Instruct_BMA_theta_0.8',
 'Qwen2.5-7B-Instruct_BMA_theta_0.9',
 'Qwen2.5-7B-Instruct_ES_theta_0.1',
 'Qwen2.5-7B-Instruct_ES_theta_0.2',
 'Qwen2.5-7B-Instruct_ES_theta_0.3',
 'Qwen2.5-7B-Instruct_ES_theta_0.4',
 'Qwen2.5-7B-Instruct_ES_theta_0.5',
 'Qwen2.5-7B-Instruct_ES_theta_0.6',
 'Qwen2.5-7B-Instruct_ES_theta_0.7',
 'Qwen2.5-7B-Instruct_ES_theta_0.8',
 'Qwen2.5-7B-Instruct_ES_theta_0.9',
 'Qwen2.5-7B-Instruct_RFA_theta_0.1',
 'Qwen2.5-7B-Instruct_RFA_theta_0.2',
 'Qwen2.5-7B-Instruct_RFA_theta_0.3',
 'Qwen2.5-7B-Instruct_RFA_theta_0.4',
 'Qwen2.5-7B-Instruct_RFA_theta_0.5',
 'Qwen2.5-7B-Instruct_RFA_theta_0.6',
 'Qwen2.5-7B-Instruct_RFA_theta_0.7',
 'Qwen2.5-7B-Instruct_RFA_theta_0.8',
 'Qwen2.5-7B-Instruct

In [ ]:
for key in loop_dict.keys():
  one_model_start_to_finish(loop_dict[key]["id1"], loop_dict[key]["theta"], tokenizer, f"{USERNAME}/{key}")

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.09G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.88G [00:00<?, ?B/s]

...

darturi/Qwen2.5-7B-Instruct_RFA_theta_0.9 || theta=0.9 || Base Models Created
darturi/Qwen2.5-7B-Instruct_RFA_theta_0.9 || theta=0.9 || LMC Vaccinated Model Created


adapter_model.safetensors:   0%|          | 0.00/323M [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

darturi/Qwen2.5-7B-Instruct_RFA_theta_0.9 || theta=0.9 || Model Saved
darturi/Qwen2.5-7B-Instruct_RFA_theta_0.9 || theta=0.9 || GPU Freed
